In [ ]:
import pandas as pd

In [ ]:
caps_dfs = {}
obitos_dfs = {}

anos = range(2014, 2024)

for ano in anos:
    caps_filename = f'CAPS{ano}.csv'
    obitos_filename = f'OBITOS{ano}.csv'
    try:
        df = pd.read_csv(caps_filename, sep=';')
        caps_dfs[f'df_caps_{ano}'] = df.dropna()
    except FileNotFoundError:
        print(f"{caps_filename} não encontrado.")
    try:
        df = pd.read_csv(obitos_filename, sep=';', encoding='latin1')
        obitos_dfs[f'df_obitos_{ano}'] = df.dropna()
    except FileNotFoundError:
        print(f"{obitos_filename} não encontrado.")
df_gini = pd.read_excel('GINI.xls')
df_ivs = pd.read_excel('IVS.xls')
df_estudo = pd.read_excel('ESTUDO.xls')
df_ppc_685 = pd.read_excel('PPC685.xls')
df_desemprego = pd.read_excel('DESEMPREGO.xls')
df_populacao = pd.read_excel('POPULACAO.xls')
df_renda = pd.read_excel('RENDA.xls')

In [ ]:
lista_caps = []
lista_obitos = []

for df_caps_item in caps_dfs:
  ano = int(caps_dfs[df_caps_item].columns[1])  # pega o nome da segunda coluna
  df_processed = caps_dfs[df_caps_item].rename(columns={caps_dfs[df_caps_item].columns[1]: "qtd_caps"})
  df_processed["ano"] = ano
  df_processed = df_processed[["UF", "ano", "qtd_caps"]]
  lista_caps.append(df_processed)

df_tratado = pd.concat(lista_caps, ignore_index=True)

df_tratado["UF"] = df_tratado["UF"].astype(int) # Converte a coluna UF para inteiro
ufs = sorted(df_tratado["UF"].unique())
anos = sorted(df_tratado["ano"].unique())

# produto cartesiano UF x ano
import itertools
grid = pd.DataFrame(list(itertools.product(ufs, anos)), columns=["UF", "ano"])
df_caps_unificado = grid.merge(df_tratado, on=["UF", "ano"], how="left")

for df_obitos_item in obitos_dfs:
  ano = int(obitos_dfs[df_obitos_item].columns[1]) # pega o nome da segunda coluna
  df_processed_obitos = obitos_dfs[df_obitos_item].rename(columns={obitos_dfs[df_obitos_item].columns[1]: "qtd_obitos"})
  df_processed_obitos["ano"] = ano
  df_processed_obitos = df_processed_obitos[["UF", "ano", "qtd_obitos"]]
  lista_obitos.append(df_processed_obitos)

df_obitos_tratado = pd.concat(lista_obitos, ignore_index=True)

df_obitos_tratado["UF"] = df_obitos_tratado["UF"].astype(int)

df_obitos_unificado = grid.merge(df_obitos_tratado, on=["UF", "ano"], how="left")

df_final = pd.merge(df_caps_unificado, df_obitos_unificado, on=['UF', 'ano'], how='outer')

# Transformar df_gini para o formato longo e mergear
df_gini_long = df_gini.melt(id_vars=['UF'], var_name='ano', value_name='gini')
df_gini_long['ano'] = df_gini_long['ano'].astype(int)
if 'gini' in df_final.columns:
    df_final = df_final.drop(columns=['gini'])
df_final = pd.merge(df_final, df_gini_long, on=['UF', 'ano'], how='left')

# df_ivs - coluna: ivs
df_ivs_long = df_ivs.melt(id_vars=['UF'], var_name='ano', value_name='ivs')
df_ivs_long['ano'] = df_ivs_long['ano'].astype(int)
if 'ivs' in df_final.columns:
    df_final = df_final.drop(columns=['ivs'])
df_final = pd.merge(df_final, df_ivs_long, on=['UF', 'ano'], how='left')

# df_estudo - coluna: anos_estudo
df_estudo_long = df_estudo.melt(id_vars=['UF'], var_name='ano', value_name='anos_estudo')
df_estudo_long['ano'] = df_estudo_long['ano'].astype(int)
if 'anos_estudo' in df_final.columns:
    df_final = df_final.drop(columns=['anos_estudo'])
df_final = pd.merge(df_final, df_estudo_long, on=['UF', 'ano'], how='left')

# df_ppc_685 - coluna: ppc_685
df_ppc_685_long = df_ppc_685.melt(id_vars=['UF'], var_name='ano', value_name='ppc_685')
df_ppc_685_long['ano'] = df_ppc_685_long['ano'].astype(int)
if 'ppc_685' in df_final.columns:
    df_final = df_final.drop(columns=['ppc_685'])
df_final = pd.merge(df_final, df_ppc_685_long, on=['UF', 'ano'], how='left')

# df_desemprego - coluna: taxa_desemprego
df_desemprego_long = df_desemprego.melt(id_vars=['UF'], var_name='ano', value_name='taxa_desemprego')
df_desemprego_long['ano'] = df_desemprego_long['ano'].astype(int)
if 'taxa_desemprego' in df_final.columns:
    df_final = df_final.drop(columns=['taxa_desemprego'])
df_final = pd.merge(df_final, df_desemprego_long, on=['UF', 'ano'], how='left')

# df_populacao - coluna: populacao
df_populacao_long = df_populacao.melt(id_vars=['UF'], var_name='ano', value_name='populacao')
df_populacao_long['ano'] = df_populacao_long['ano'].astype(int)
df_populacao_long['populacao'] = df_populacao_long['populacao'] * 1000
if 'populacao' in df_final.columns:
    df_final = df_final.drop(columns=['populacao'])
df_final = pd.merge(df_final, df_populacao_long, on=['UF', 'ano'], how='left')

# df_renda: coluna: renda_mdpc
df_renda_long = df_renda.melt(id_vars=['UF'], var_name='ano', value_name='renda_mdpc')
df_renda_long['ano'] = df_renda_long['ano'].astype(int)
if 'renda_mdpc' in df_final.columns:
    df_final = df_final.drop(columns=['renda_mdpc'])
df_final = pd.merge(df_final, df_renda_long, on=['UF', 'ano'], how='left')

df_final = df_final.loc[:,~df_final.columns.duplicated()].copy()

display(df_final.head())

,UF,ano,qtd_caps,qtd_obitos,gini,ivs,anos_estudo,ppc_685,taxa_desemprego,populacao,renda_mdpc
0,11,2014,3.0,84.0,0.443,0.176,7.70,580285,4.25,1668000,1230.283078
1,11,2015,5.0,109.0,0.462,0.206,8.08,630481,5.65,1688000,1224.803032
2,11,2016,4.0,103.0,0.465,0.193,8.00,647250,8.03,1708000,1194.016683
3,11,2017,5.0,113.0,0.447,0.189,8.40,600513,8.28,1727000,1250.239373
4,11,2018,3.0,125.0,0.496,0.197,8.60,585968,9.23,1747000,1413.383040


In [ ]:
# Calcular a taxa de suicídio por 100.000 habitantes
df_final['taxa_suicidio'] = (df_final['qtd_obitos'] / df_final['populacao']) * 100000

# Remover as colunas qtd_obitos e populacao
df_final = df_final.drop(columns=['qtd_obitos', 'populacao'])

display(df_final.head())

,UF,ano,qtd_caps,gini,ivs,anos_estudo,ppc_685,taxa_desemprego,renda_mdpc,taxa_suicidio
0,11,2014,3.0,0.443,0.176,7.70,580285,4.25,1230.283078,5.035971
1,11,2015,5.0,0.462,0.206,8.08,630481,5.65,1224.803032,6.457346
2,11,2016,4.0,0.465,0.193,8.00,647250,8.03,1194.016683,6.030445
3,11,2017,5.0,0.447,0.189,8.40,600513,8.28,1250.239373,6.543138
4,11,2018,3.0,0.496,0.197,8.60,585968,9.23,1413.383040,7.155123


In [ ]:
df_final.to_csv('df_unificado.csv', index=False)